In [1]:
import pandas as pd

In [6]:
from bs4 import BeautifulSoup
import requests
from lxml import html
import json
import numpy as np

# Get Players

In [7]:
dump = requests.get('https://fantasy.premierleague.com/drf/bootstrap')
load = dump.text

In [8]:
json_dump = json.loads(load)

In [10]:
datapoints = ['id', 'web_name', 'first_name', 'total_points', 'event_points', 'now_cost','cost_change_start', 
              'selected_by_percent', 'form','ep_next', 
              'chance_of_playing_next_round','minutes', 'bonus', 'goals_scored', 'assists', 'clean_sheets', 
              'goals_conceded', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'ea_index', 'news']

player_list = []

for player in range(0, len(json_dump['elements'])):
    stats = [json_dump['elements'][player][x] for x in datapoints]
    player_list.append(stats)
    
player_df = pd.DataFrame(player_list)

# Rename columns

player_df.columns = datapoints

# Get Positions

In [11]:
players = requests.get('https://fantasy.premierleague.com/player-list/')
totals = players.text
total_soup = BeautifulSoup(totals)

In [12]:
#Get all player names

player_names = total_soup.find_all('td')[::4]

players = []

for player in player_names:
    name = player.get_text()
    players.append(name)

player_name_df = pd.DataFrame({"Name": players})

In [13]:
# Positions for first and last player in each field

for i, e in enumerate(player_name_df['Name']):
    if e == 'De Gea':   
        gk=i
    if e == 'Azpilicueta':   
        defe=i
    if e == 'Salah':   
        mid=i
    if e == 'Kane':   
        att=i

In [14]:
#Populate df with positions
player_name_df['position']=''


player_name_df.iloc[gk:(defe)]['position']='Goalkeeper'
player_name_df.iloc[defe:(mid)]['position']='Defender'
player_name_df.iloc[mid:(att)]['position']='Midfielder'
player_name_df.iloc[att:]['position']='Attacker'

/Users/alexis.becker/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/alexis.becker/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/alexis.becker/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [15]:
fpl_players =  pd.merge(player_name_df, player_df, left_on='Name', right_on='web_name')

In [16]:
# Deal with duplicates

duplicats = fpl_players['Name'].value_counts()>1

dup_names = fpl_players['Name'].isin(['McCarthy','Ward','Bennett','Davies','Gray','Long','Simpson','Reid','Sánchez','McCartyh','Stephens', 'Pereira', 'Murphy', 'Williams'])


fpl_players.drop(fpl_players[dup_names].index, inplace=True)

/Users/alexis.becker/anaconda/envs/py27/lib/python2.7/site-packages/pandas/core/algorithms.py:410: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  f = lambda x, y: htable.ismember_object(x, values)


# Optimal Choice

In [17]:
import pulp

In [18]:
#Make variables

#Player variables by position
threat = fpl_players["threat"]
influence = fpl_players[['id',"influence"]]

players = range(0, len(fpl_players))


gks = fpl_players[fpl_players['position']=='Goalkeeper']
defs = fpl_players[fpl_players['position']=='Defender']
mds = fpl_players[fpl_players['position']=='Midfielder']
ats = fpl_players[fpl_players['position']=='Attacker']

rgks=range(0, len(gks))
rdefs=range(0, len(defs))
rmds=range(0, len(mds))
rats=range(0, len(ats))

In [19]:
# Dummy GK id

test = gks['id']

In [25]:
# Formula to evaluate

def total_influence(player):
    """Find the happiness of the table - 
    by calculating the combined influence between the players"""
    return sum(fpl_players[fpl_players['id']==player[0]]['influence'].astype(float),fpl_players[fpl_players['id']==player[-1]]['influence'].astype(float))

In [20]:
#Set constraint of variables
max_gks = 2
max_money = 1000

#create list of all possible tables https://www.ce.teiep.gr/e-class/modules/document/file.php/216/PAPERS/2009.%20Pulp%20For%20Python%20Programmers.pdf
possible_gks = [tuple(c) for c in pulp.allcombinations(gks['id'], max_gks)]

x = pulp.LpVariable('x', upBound=max_money, cat='Continuous')

# Look for all players used
y = pulp.LpVariable.dicts('keeper_used', players, lowBound=0,upBound=1, cat=pulp.LpInteger)

#Indicator variable, showing if player been used
g = pulp.LpVariable.dicts('keeper_used', test, lowBound=0,upBound=1, cat=pulp.LpInteger)
#create a binary variable to state that a player combo is used
gu = pulp.LpVariable.dicts('gks', possible_gks, lowBound = 0, upBound = 1,cat = pulp.LpInteger)
#d = pulp.LpVariable.dicts('d', rdefs, 0,1, pulp.LpInteger)
#m = pulp.LpVariable.dicts('m', rmds, 0,1, pulp.LpInteger)
#a = pulp.LpVariable.dicts('a', rats, 0,1, pulp.LpInteger)

In [26]:
#Create optimisation problem

player_choice = pulp.LpProblem("Player choice model", pulp.LpMaximize)

player_choice += pulp.lpSum([ y[player]*
                             #influence.loc[influence['id']==player]['influence'].astype(float) 
                             float(influence.iloc[player]['influence']) 
                             for player in players])

In [29]:
#Create Constraints

#Only 2 players
player_choice += sum([y[player] for player in players]) <= 2

In [24]:
seating_model += sum([x[table] for table in possible_tables]) <= max_tables,

1

In [35]:
#Solving 
player_choice.solve()

for v in player_choice.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)
    else:
        continue

('keeper_used_211', '=', 1.0)
('keeper_used_392', '=', 1.0)


# Iteration 2.0